In [146]:
import pandas as pd
import numpy as np
import itertools

In [147]:
def load_csv(file_path:type=str):
    '''
    :param file_path:type=str, the file's whole path and name, the file must be a csv file
    :return vertexs: type=list all the vertexs' 3d coordinates in a list, coordinate is in list type
    '''
    vertexs = []
    with open(file_path,'r') as file:
        for line in file:
            if line.strip().startswith('vertex'):
                parts = line.split()
                x,y,z = float(parts[1]), float(parts[2]), float(parts[3])
                coordinate = [x,y,z]
                if coordinate not in vertexs:
                    vertexs.append([x,y,z])    
    return vertexs

In [148]:
def get_centroid(vertexs:type=list):
    '''
    to calculate all(in the given list) the vertexs' geometric centroid, return a centroid in list
    '''
    num_vertexs = len(vertexs)
    x_sum = 0
    y_sum = 0
    z_sum = 0
    vertexsi=[]
    for i in range(num_vertexs):
        vertexsi = vertexs[i]
        temp_x = vertexsi[0]
        temp_y = vertexsi[1]
        temp_z = vertexsi[2]
        x_sum = x_sum+temp_x
        y_sum = y_sum+temp_y
        z_sum = z_sum+temp_z
        
    x_centroid = x_sum/num_vertexs
    y_centroid = y_sum/num_vertexs
    z_centroid = z_sum/num_vertexs
    centroid  = [x_centroid, y_centroid, z_centroid]
    return centroid

In [149]:
def origin_transformation(centroid:type=list,vertexs:type=list):
    '''
    transform the coordinate system's origin to the given centroid
    in the OPD, it is the the Geometric Center of all vertexs
    
    '''
    df_vertexs = pd.DataFrame(vertexs,columns=['x', 'y', 'z'])
    a = centroid[0]
    b = centroid[1]
    c = centroid[2]
    df_vertexs['x'] = df_vertexs['x'] - a
    df_vertexs['y'] = df_vertexs['y'] - b
    df_vertexs['z'] = df_vertexs['z'] - c
    
    vertexs_list = df_vertexs.values.tolist()
    return vertexs_list

In [150]:
def get_projection_on_plane(v_list:type=list,n_list:type=list):
    '''
    n_list is the plane's normal vector in list type, 
    v_list is the vector that to do the projection to the plane
    return vector n's projection vector and the projection vector's length
    '''
    v = np.array(v_list,dtype=float)
    n = np.array(n_list,dtype=float)
    n_normalized = n / np.linalg.norm(n)
    projection_on_normal = np.dot(v, n_normalized) * n_normalized
    perpendicular_component = v - projection_on_normal
    projection_length = np.linalg.norm(perpendicular_component)
    
    if projection_length != 0:
        projection_direction = perpendicular_component 
    else:
        projection_direction = np.zeros_like(perpendicular_component)
    
    return projection_direction,projection_length #projection_direction in array type

In [151]:
def set_axes(vertexs_list):
    '''
    choose the farest point(to the origin) as the z axes,
    and then project all points in to the xoy plane, choose the longest projection's direction as the y axes
    and then tranform al the points' coordiante system into the new coordiante system
    return the points's list (coordiantes in new system)
    
    '''
    
    farest_distance = 0 # get farest distance vertex as the z axes
    farest_vertex = []

    for i in range(len(vertexs_list)):
        vertex_i = vertexs_list[i]
        vertex_i_array = np.array(vertex_i)
        i_norm = np.linalg.norm(vertex_i_array)
        if i_norm > farest_distance:
            farest_distance = i_norm
            farest_vertex = vertex_i
        else:continue
    
    projection = 0
    projection_direction = [] #longest projection on xoy plane as the y axes
    
    other_vertexs = vertexs_list.copy()
    other_vertexs.remove(farest_vertex)
    for j in range(len(other_vertexs)):
        vertex_j = other_vertexs[j]
        
        projection_j_direction,projection_j_length = get_projection_on_plane(vertex_j,farest_vertex)
        
        if projection_j_length > projection:
            projection = projection_j_length
            projection_direction = list(projection_j_direction)
        else:
            continue

    x_vector_array = np.cross(projection_direction,farest_vertex) #get x axes's direction
          
    x_vector_norm = np.linalg.norm(x_vector_array)
    x_vector = list(x_vector_array)

    z_direction = farest_vertex
    y_direction = projection_direction
    
    x_direction = x_vector
    
    point11 = farest_vertex/np.linalg.norm(farest_vertex)
    point12 = [0,0,1]
    
    point21 = projection_direction/projection
    point22 = [0,1,0]
    
    point31 = x_vector/x_vector_norm
    point32 = [1,0,0]
    
    X1 = np.array([point31,point21,point11])
    X2 = np.array([point32,point22,point12])
    X1_transpose = X1.T
    X2_transpose = X2.T

    X1_T_inv = np.linalg.inv(X1_transpose)
    M = np.dot(X2_transpose, X1_T_inv) #get the transformation matrix between two coordiante system
    
    vertexs_after_transfomation = [] #points' coodinate transformation between two system(only xes rotation)
    for i in range(len(vertexs_list)):
        vertex_i = vertexs_list[i]
        array_vertex_transpose = np.array(vertex_i).T
        transformed_vertex_i = np.dot(M,array_vertex_transpose)
        transformed_vertex_i_list = list(transformed_vertex_i.T)
        vertexs_after_transfomation.append(transformed_vertex_i_list)
    return vertexs_after_transfomation 

In [172]:
def regional_division(vertexs_after_transformation:type=list):
    '''
    divide poionts into 8 quadrants return a region list with 8 region(contains all vertexs in this region,list type) as the element
    :param vertexs_after_transformation type=list, points's coordinates after re-choose the x,y,z axis
    '''
    region1 = [] #x>0,y>0,z>0
    region2 = [] #x>0,y>0,z<0
    region3 = [] #x>0,y<0,z>0
    region4 = [] #x>0,y<0,z<0
    region5 = [] #x<0,y>0,z>0
    region6 = [] #x<0,y>0,z<0
    region7 = [] #x<0,y<0,z>0
    region8 = [] #x<0,y<0,z<0
    for i in range(len(vertexs_after_transformation)):
        vertex_i = vertexs_after_transformation[i]
        ix = vertex_i[0]
        iy = vertex_i[1]
        iz = vertex_i[2]
        if ix>0:
            if iy>0:
                if iz>0:
                    region1.append(vertex_i)
                elif iz<0:
                    region2.append(vertex_i)
            elif iy<0:
                if iz>0:
                    region4.append(vertex_i)
                elif iz<0:
                    region3.append(vertex_i)
        elif ix<0:
            if iy>0:
                if iz>0:
                    region5.append(vertex_i)
                elif iz<0:
                    region6.append(vertex_i)
            elif iy<0:
                if iz>0:
                    region8.append(vertex_i)
                elif iz<0:
                    region7.append(vertex_i)
    region_list = [region1,region2,region3,region4,region5,region6,region7,region8]
    
    for i in range(8):
        temp_region = region_list[i]
        len_region = len(temp_region)
        if len_region == 0:
            print(f"the region{i} has no point")
        
    #print('1',region1,'2',region2,'3',region3,'4',region4,'5',region5,'6',region6,'7',region7,'8',region8)
    
    return region_list

def OPD_calculation(centroid_list:type=list,m:type=int,n:type=int):
    '''
    the sum part in opd function
    '''
    N = len(centroid_list)
    P = itertools.permutations(np.arange(N),4)
    
    coord = []
    
    for i in range(len(centroid_list)):
        array_i = np.array(centroid_list[i])
        coord.append(array_i)
                   
                   
    for kk in P:
        r_ij = coord[kk[0]]-coord[kk[1]]
        r_kl = coord[kk[2]]-coord[kk[3]]
        r_il = coord[kk[0]]-coord[kk[3]]
        r_jk = coord[kk[1]]-coord[kk[2]]
        r_ij_mag = np.linalg.norm(r_ij)
        r_kl_mag = np.linalg.norm(r_kl)
        r_il_mag = np.linalg.norm(r_il)
        r_jk_mag = np.linalg.norm(r_jk)        
        
        G_p_up = np.dot(np.cross(r_ij, r_kl),r_il)*(np.dot(r_ij, r_jk))*(np.dot(r_jk, r_kl))
        G_p_down = ((r_ij_mag*r_jk_mag*r_kl_mag)**n)*((r_il_mag)**m)
        G_p = G_p_up/G_p_down
  
    
        yield G_p

def opd(centroid_list,m,n):
    '''
    use the 8 sub centroids to calculate OPD value, m,n can be selfly setted
    '''
    N = len(centroid_list)
    OPD = (1/3)*sum(OPD_calculation(centroid_list,m,n))
    return OPD

def file_to_opd(file_path,m,n): 
    '''
    just set the file's whole name in str and m,n's value in interger, the function will use the functions above to return a OPD value
    '''
    vertexs = load_csv(file_path)  # get all vertexs' coordinate 
    centroid = get_centroid(vertexs) #get centroid 
    origin_translated_vertexs = origin_transformation(centroid,vertexs)  #move coordinate's origin to the system's centroid
    
    after_set = set_axes(origin_translated_vertexs) #set new coordinate system 
    reformed_coordiante_vertexs = regional_division(after_set) # separate into 8 sectionsqit 
    
    sub_centroid = []
    for i in range(8): #calculate the 8 sub-centroids to put into OPD calculation
        temp_region = reformed_coordiante_vertexs[i]
        
        temp_centroid = get_centroid(temp_region)
        sub_centroid.append(temp_centroid)
    
    opd_value = opd(sub_centroid,m,n)
    return opd_value

In [174]:
file_to_opd('your_file_whole_path.csv',1,2)

-0.3978095005404761